In [1]:
from pyspark.sql import functions as F

df = spark.table("lh_australia_energy.energy_finance_analysis")

df.createOrReplaceTempView("energy")

df.printSchema()

StatementMeta(, f98d0921-69f8-4fa0-9a96-61f57a175f51, 3, Finished, Available, Finished)

root
 |-- REGION: string (nullable = true)
 |-- SETTLEMENTDATE: timestamp (nullable = true)
 |-- TOTALDEMAND: double (nullable = true)
 |-- RRP: double (nullable = true)
 |-- PERIODTYPE: string (nullable = true)
 |-- JoinDate: date (nullable = true)
 |-- Date: date (nullable = true)
 |-- AUDUSD_X: double (nullable = true)
 |-- BHP_AX: double (nullable = true)
 |-- NG_F: double (nullable = true)



In [2]:
query_hourly = """
    SELECT
        HOUR(SETTLEMENTDATE) as Hour,
        ROUND(AVG(TOTALDEMAND), 0) as Avg_Demand,
        ROUND(AVG(RRP), 2) as Avg_Price,
        ROUND(AVG(AUDUSD_X), 4) as Avg_FX
    FROM energy
    GROUP BY HOUR(SETTLEMENTDATE)
    ORDER BY Avg_Demand
"""

df_hourly = spark.sql(query_hourly)
display(df_hourly)

StatementMeta(, f98d0921-69f8-4fa0-9a96-61f57a175f51, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4dfcfa6e-816e-4edd-9deb-267c1fb34ed4)

In [3]:
query_monthly = """
    SELECT
        MONTH(SETTLEMENTDATE) as Month,
        HOUR(SETTLEMENTDATE) as Hour,
        ROUND(AVG(TOTALDEMAND), 0) as Avg_Demand,
        ROUND(AVG(RRP), 2) as Avg_Price,
        ROUND(AVG(AUDUSD_X), 4) as Avg_FX
    FROM energy
    WHERE MONTH(SETTLEMENTDATE) IN (1, 11)
    GROUP BY MONTH(SETTLEMENTDATE), HOUR(SETTLEMENTDATE)
    ORDER BY Avg_Price
"""

df_monthly = spark.sql(query_monthly)
display(df_monthly)

StatementMeta(, f98d0921-69f8-4fa0-9a96-61f57a175f51, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b9782811-0719-4f6c-b3f3-c97e4e8f1d8f)

In [4]:
query_negative = """
    SELECT
        HOUR(SETTLEMENTDATE) as Hour,
        COUNT(*) as Total_Count,
        SUM(CASE WHEN RRP < 0 THEN 1 ELSE 0 END) as Negative_Count
    FROM energy
    GROUP BY HOUR(SETTLEMENTDATE)
    ORDER BY Hour
"""

df_negative = spark.sql(query_negative)
display(df_negative)


StatementMeta(, f98d0921-69f8-4fa0-9a96-61f57a175f51, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b13bdb7b-2158-4547-aeb7-d3e5b3bfc5bb)

In [5]:
query_fx = """
    SELECT

        CASE
            WHEN AUDUSD_X >= 0.64 THEN 'High FX'
            ELSE 'Low FX'
        END as FX_Group,

        ROUND(AVG(RRP), 2) as Avg_Price,
        ROUND(AVG(TOTALDEMAND), 0) as Avg_Demand
    
    FROM energy
    GROUP BY 
        CASE
            WHEN AUDUSD_X >= 0.64 THEN 'High FX'
            ELSE 'Low FX'
        END 
    ORDER BY FX_Group
"""

df_fx = spark.sql(query_fx)
display(df_fx)

StatementMeta(, f98d0921-69f8-4fa0-9a96-61f57a175f51, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f028dd21-e20b-4e5d-98d6-b1c61687e87a)

In [6]:
df = spark.table("lh_australia_energy.energy_finance_analysis")
df_report = df.withColumn("Hour", F.hour("SETTLEMENTDATE")).withColumn("FX_Group", F.when(F.col("AUDUSD_X") >= 0.64, "High FX").otherwise("Low FX"))

df_report.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').saveAsTable("lh_australia_energy.energy_finance_analysis")

StatementMeta(, f98d0921-69f8-4fa0-9a96-61f57a175f51, 8, Finished, Available, Finished)